In [1]:
from woocommerce import API
import os
from dotenv import load_dotenv

load_dotenv()

woocommerce_url = os.getenv('URL')
woocommerce_key = os.getenv('KEY')
woocommerce_secret = os.getenv('SECRET')

wc_api = API(
    url=woocommerce_url,
    consumer_key=woocommerce_key,
    consumer_secret=woocommerce_secret,
    version='wc/v3'
)

In [2]:
product_sku = 'S2044-449-1'
product = wc_api.get('products', params={'sku': product_sku}).json()[0]

In [79]:
existing_attributes = product["attributes"]

In [81]:
new_attribute = {
    'id': 7,
    'name': 'sizechart',
    'position': 5,
    'visible': True,
    'variation': False,
    'options': ['SAUMFTW']
}
modified_attribute = {'id': 2,
    'options': ['Saucony Originals']
}

In [82]:
for attribute in existing_attributes:
    if attribute['id'] == modified_attribute['id']:
        attribute.update(modified_attribute)

In [83]:
existing_attributes.append(new_attribute)


In [85]:
product_id = product['id']

In [86]:
payload = {"attributes": existing_attributes}

In [ ]:
wc_api.put(f"products/{product_id}", payload).json()

In [ ]:
existing_attributes